<a href="https://colab.research.google.com/github/ashamril/Text-Analytics/blob/master/requestsURL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import requests
import calendar
import re
from collections import Counter
from wordcloud import WordCloud
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict  
from pandas import DataFrame
import plotly.express as px

In [2]:
# URL = "https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2021/3/18"

# dfAll = pd.DataFrame()
# dfAllNegeri = pd.DataFrame()
# response = requests.get(URL)
# if response.status_code == 200:
#   html_text = response.text
#   soup = BeautifulSoup(html_text)
#   tables = soup.findAll('table')

#   for table in tables:
#     if "Pusat" in table.text:
#       selected_negeri = table
#       dfNegeri = pd.read_html(str(selected_negeri), header=None)[0]
#       dfHarga = pd.read_html(str(selected_table), header=None)[0]
#       dfHarga["Negeri"] = dfNegeri
#       dfAll = pd.concat([dfAll, dfHarga])
       
#     else:
#       print("Link 404! {}".format(URL))

In [4]:
URL = "https://sdvi2.fama.gov.my/price/direct/price/daily_commodityRpt.asp?Pricing=A&LevelCd=03&PricingDt=2021/3/18"

dfAllNegeri = pd.DataFrame()

response = requests.get(URL)
html_text = response.text
soup = BeautifulSoup(html_text)
tables = soup.findAll('table')

for table in tables[1:]:
  if "Pusat" in table.text:
    dfNegeri = pd.read_html(str(table), header=None)[0]
    dfAllNegeri = pd.concat([dfAllNegeri, dfNegeri])

#dfAllNegeri = dfAllNegeri[dfAllNegeri[0].str.contains("Pusat")]
lstAllNegeri = dfAllNegeri.values.tolist()

In [14]:
i = 0
dfAllHarga = pd.DataFrame()
for table in tables[1:]:
  if "Nama Varieti" in table.text:
    if i < len(lstAllNegeri):
      dfHarga = pd.read_html(str(table), header=None)[0]
      dfHarga = dfHarga[~dfHarga[0].str.contains("Varieti")]
      dfHarga["Pusat"] = str(lstAllNegeri[i])
      dfAllHarga = pd.concat([dfAllHarga, dfHarga])
      i += 1

dfAllHarga.columns=["Nama Varieti", "Gred", "Unit", "HargaTinggi", "HargaPurata", "HargaRendah", "Pusat"]
dfAllHarga['Pusat'] = dfAllHarga['Pusat'].str.replace("\['Pusat : ", "")
dfAllHarga['Pusat'] = dfAllHarga['Pusat'].str.replace("'\]", "")
dfAllHarga = dfAllHarga.reset_index()
dfAllHarga = dfAllHarga.drop('index', axis = 1) 
dfAllHarga.head()

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat
0,AYAM HIDUP,F.A.Q,KILOGRAM,7.30,7.30,7.30,"JOHOR BAHRU, JOHOR"
1,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,8.60,8.60,8.60,"JOHOR BAHRU, JOHOR"
2,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,6.00,5.50,5.00,"JOHOR BAHRU, JOHOR"
3,BAWANG BESAR MERAH (IMPORT),F.A.Q,KILOGRAM,6.00,5.50,5.00,"JOHOR BAHRU, JOHOR"
4,BAWANG BESAR MERAH (INDIA),F.A.Q,KILOGRAM,6.00,5.50,5.00,"JOHOR BAHRU, JOHOR"


In [15]:
dfAllHarga[dfAllHarga['Pusat'].str.contains("selangor", na=False, case=False)]

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat
337,AYAM HIDUP,F.A.Q,KILOGRAM,6.50,6.50,6.50,"GOMBAK, SELANGOR"
338,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,7.80,7.65,7.50,"GOMBAK, SELANGOR"
339,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,4.00,4.00,4.00,"GOMBAK, SELANGOR"
340,BAWANG BESAR MERAH (IMPORT),F.A.Q,KILOGRAM,4.00,3.75,3.50,"GOMBAK, SELANGOR"
341,BAWANG BESAR MERAH (INDIA),F.A.Q,KILOGRAM,4.50,4.25,4.00,"GOMBAK, SELANGOR"
...,...,...,...,...,...,...,...
451,TEMBIKAI MERAH TANPA BIJI,F.A.Q,KILOGRAM,3.00,3.00,3.00,"PETALING, SELANGOR"
452,TERUNG PANJANG/PEPARU (TANAH RENDAH),F.A.Q,KILOGRAM,5.00,5.00,5.00,"PETALING, SELANGOR"
453,TIMUN HIJAU,F.A.Q,KILOGRAM,2.00,2.00,2.00,"PETALING, SELANGOR"
454,TOMATO (TANAH TINGGI),F.A.Q,KILOGRAM,4.50,4.50,4.50,"PETALING, SELANGOR"


In [16]:
dfAllHarga.loc[dfAllHarga['Pusat'] == "KOTA SETAR, KEDAH"] 

,Nama Varieti,Gred,Unit,HargaTinggi,HargaPurata,HargaRendah,Pusat
38,AYAM PROSES STANDARD,F.A.Q,KILOGRAM,8.50,8.50,8.00,"KOTA SETAR, KEDAH"
39,AYAM PROSES SUPER,F.A.Q,KILOGRAM,9.50,9.50,9.00,"KOTA SETAR, KEDAH"
40,BAWANG BESAR KUNING/HOLLAND,F.A.Q,KILOGRAM,5.00,5.00,4.80,"KOTA SETAR, KEDAH"
41,BAWANG BESAR MERAH (IMPORT),F.A.Q,KILOGRAM,5.50,5.00,5.00,"KOTA SETAR, KEDAH"
42,BAWANG MERAH/KECIL (IMPORT),F.A.Q,KILOGRAM,8.00,7.00,6.00,"KOTA SETAR, KEDAH"
43,BAWANG MERAH/KECIL (THAI),F.A.Q,KILOGRAM,10.00,9.00,8.00,"KOTA SETAR, KEDAH"
44,BAWANG PUTIH (CHINA),F.A.Q,KILOGRAM,7.00,7.00,6.00,"KOTA SETAR, KEDAH"
45,BAYAM,F.A.Q,KILOGRAM,6.00,5.00,4.50,"KOTA SETAR, KEDAH"
46,BELIMBING (B10),F.A.Q,KILOGRAM,8.00,7.00,6.00,"KOTA SETAR, KEDAH"
47,BETIK SEKAKI,F.A.Q,KILOGRAM,4.50,4.50,4.00,"KOTA SETAR, KEDAH"
